<a href="https://colab.research.google.com/github/EmKelsall/ADS2002---Catheter-Placement-Project/blob/main/SVMmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ADS2002-Project-CatheterPlacement



Mounted at /content/drive
/content/drive/MyDrive/ADS2002-Project-CatheterPlacement


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from PIL import Image

filenames = os.listdir("Data/train")
train_imgs = np.empty((0,600, 600))
FileIDs = []
lst = []

count = 0
for file in filenames:
  if count == 500:
    break

  #Making sure the images are the same pixel size
  img = Image.open(f"/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train/{file}")
  img = img.resize((600,600))
  img = np.array(img)

  #add extra dimension so we can concatenate
  img = np.expand_dims(img,(0))

  #Save filename for later
  FileIDs.append(file)

  #adding to train_imgs array
  train_imgs = np.concatenate((train_imgs, img), axis = 0)

  count += 1

# Now, 'cvc_images' contains your preprocessed CVC catheter images, and 'cvc_labels' contains the placement labels (Abnormal, Normal, Borderline).

In [ ]:
#extracting just the CVC labels
train_data = pd.read_csv("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train.csv")
CVC_train = train_data[["StudyInstanceUID","CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
CVC_train.set_index('StudyInstanceUID', inplace=True)
CVC_train.head()


#cleaning up the labels
UID = []
for i in range(len(FileIDs)):
  UID.append(FileIDs[i].replace(".jpg",""))


In [ ]:

train_labels = np.empty((0,3))
data = []
for ID in UID:
    data.append(CVC_train.loc[ID])

train_labels_df = pd.DataFrame(data)

train_labels_df = train_labels_df.reindex(UID)

train_labels = train_labels_df.values

train_labels



array([[0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [ ]:
# Assuming you have a list 'labels' with corresponding labels for each image (Abnormal, Normal, Borderline)
# Example:
# labels = ["CVC - Abnormal", "CVC - Normal", "CVC - Borderline"]  # Make sure this list matches your dataset

from sklearn.model_selection import train_test_split

# Flatten the images in train_imgs
train_imgs_flattened = train_imgs.reshape(train_imgs.shape[0], -1)
train_labels_flattened = train_labels.reshape(train_labels.shape[0], -1)
train_labels_flattened = train_labels.argmax(axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(train_imgs_flattened, train_labels_flattened, test_size = 0.2, random_state=42)


# Now, your SVM classifier is trained and ready for predictions.

# You can use svm_classifier.predict(new_data) to make predictions on new data.


In [ ]:
# Import the SVM classifier from scikit-learn
from sklearn import svm

# Initialize an SVM classifier
svm_classifier = svm.SVC(kernel='linear')  # You can choose the kernel type you prefer

# Train the SVM model with your data and labels
svm_classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [ ]:
# Make predictions on the test set

from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_classifier.predict(X_test)

# Evaluate the model using accuracy and classification report
accuracy = accuracy_score(Y_test, y_pred)
classification_rep = classification_report(Y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Accuracy: 0.44
Classification Report:
              precision    recall  f1-score   support

           0       0.11      0.08      0.10        12
           1       0.32      0.23      0.27        35
           2       0.53      0.66      0.59        53

    accuracy                           0.44       100
   macro avg       0.32      0.32      0.32       100
weighted avg       0.41      0.44      0.42       100

